In [1]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 128.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [boto3]32m1/4 [botocore]
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -r requirements.txt

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ezihh3qe
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ezihh3qe
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 107.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 86.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 127.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 91.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 131.2 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import torch
import open_clip
caption_model = torch.load('caption_features_flickr8k.pt')
image_model = torch.load('image_features_flickr8k.pt')

In [7]:
image_names = list(image_model.keys())
image_embs = torch.stack(list(image_model.values()))  # shape: [N_img, 512]
caption_embs = torch.stack([cap[2] for cap in caption_model])  # shape: [N_cap, 512]
caption_img_names = [cap[0] for cap in caption_model]


In [8]:
caption_embs = caption_embs.squeeze(1)
image_embs = image_embs.squeeze(1)


In [9]:

print("caption_embs shape:", caption_embs.shape)   # should be [num_captions, 512]
print("image_embs shape:", image_embs.shape)       # should be [num_images, 512]

print("caption_embs[0] shape:", caption_embs[0].shape)  # should be [512]
print("caption_embs[0].unsqueeze(0) shape:", caption_embs[0].unsqueeze(0).shape)  # [1,512]

caption_embs shape: torch.Size([40455, 512])
image_embs shape: torch.Size([8091, 512])
caption_embs[0] shape: torch.Size([512])
caption_embs[0].unsqueeze(0) shape: torch.Size([1, 512])


In [24]:
# Text to image Recall

import torch.nn.functional as F


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

image_embs = image_embs.to(device)
caption_embs = caption_embs.to(device)

def text_to_image_recall(image_embs, caption_embs, caption_img_names, image_names, K=10):
    recalls = 0
    total = len(caption_embs)


    for i in range(total):
        sims = F.cosine_similarity(caption_embs[i].unsqueeze(0), image_embs).squeeze()
        topk = sims.topk(K).indices
        
        retrieved_imgs = [image_names[j] for j in topk]
        true_img = caption_img_names[i]

        if true_img in retrieved_imgs:
            recalls += 1

    print(f"Recall@{K}: {recalls/total:.4f}")
    return recalls / total

Using device: cuda


In [25]:
text_to_image_recall(image_embs, caption_embs, caption_img_names, image_names, K=1)

Recall@1: 0.2982


0.29815844765789146

In [26]:
text_to_image_recall(image_embs, caption_embs, caption_img_names, image_names, K=5)

Recall@5: 0.5331


0.5330861450994933

In [27]:
text_to_image_recall(image_embs, caption_embs, caption_img_names, image_names, K=10)

Recall@10: 0.6324


0.6323816586330491

In [31]:
# Image to text Recall

import torch.nn.functional as F


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

image_embs = image_embs.to(device)
caption_embs = caption_embs.to(device)

def image_to_text_recall(image_embs, caption_embs, caption_img_names, image_names, K=10):
    recalls = 0
    total = len(image_embs)


    for i in range(total):
        sims = F.cosine_similarity( image_embs[i].unsqueeze(0), caption_embs).squeeze()
        topk = sims.topk(K).indices
        
        retrieved_captions = [caption_img_names[j] for j in topk]
        true_caption = image_names[i]

        if true_caption in retrieved_captions:
            recalls += 1

    print(f"Recall@{K}: {recalls/total:.4f}")
    return recalls / total

Using device: cuda


In [32]:
image_to_text_recall(image_embs, caption_embs, caption_img_names, image_names, K=1)

Recall@1: 0.4726


0.47262390310221236

In [33]:
image_to_text_recall(image_embs, caption_embs, caption_img_names, image_names, K=5)

Recall@5: 0.7051


0.7051044370287974

In [34]:
image_to_text_recall(image_embs, caption_embs, caption_img_names, image_names, K=10)

Recall@10: 0.7908


0.7907551600543814